In [1]:
import pandas as pd
import glob
import regex as re
import querygen as qg

In [2]:
files = glob.glob("results/*.csv")
dfs = [pd.read_csv(f, index_col=0) for f in files]
files

['results/eval_results_DNB_zeroshot_NousResearch-Hermes-3-Llama-3.2-3B-GGUF.csv',
 'results/eval_results_DBpedia_zeroshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_UniProt_oneshot_NousResearch-Hermes-3-Llama-3.2-3B-GGUF.csv',
 'results/eval_results_DNB_zeroshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_DBpedia_oneshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_BTO_zeroshot_NousResearch-Hermes-3-Llama-3.2-3B-GGUF.csv',
 'results/eval_results_BTO_oneshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_UniProt_zeroshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_UniProt_oneshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_BTO_zeroshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_DBpedia_zeroshot_NousResearch-Hermes-3-Llama-3.2-3B-GGUF.csv',
 'results/eval_results_DNB_oneshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv

In [3]:
dfs

[Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
       f1_score  precision    recall   ged  normed_ged  \
 0     0.000000   0.000000  0.000000   6.0   -0.200000   
 1     0.000000   0.000000  0.000000   6.0   -0.200000   
 2     0.800000   1.000000  0.666667   3.0    0.400000   
 3     0.800000   1.000000  0.666667   3.0    0.400000   
 4     0.000000   0.000000  0.000000  45.0   -0.022727   
 ...        ...        ...       ...   ...         ...   
 4921  0.000000   0.000000  0.000000   9.0    0.000000   
 4922  0.285714   0.500000  0.200000   9.0    0.000000   
 4923  0.888889   1.000000  0.800000   4.0    0.555556   
 4924  0.000000   0.000000  0.000000  13.0    0.000000   
 4925  0.833333   0.714286  1.000000   6.0    0.538462   
 
                                                response  \
 0     The protein is a part of a larger protein doma...   
 1     The protein is a part of a larger protein doma...   
 2     The protein is a part of a larg

In [4]:

def graph_from_erl(erl: qg.EnrichedEntitiesRelations):
    G = qg.nx.DiGraph()
    for node in erl.entities:
        G.add_node(node.identifier, label=node.type)
    for link in erl.relations:
        G.add_edge(
            link.entity,
            link.target,
            weight=link.link.instance_count,
            label=link.relation,
        )
    return G

In [5]:
for f, df in zip(files, dfs):
    # data=re.findall(r"results/eval_results_(\w+)_.*", f)[0]
    # prompt_type=re.findall(r"results/eval_results_(\w+)_.*", f)[0]
    model = re.findall(r"results/eval_results_\w+_(.+)\.csv", f)[0]
    # df['prompt_type'] = prompt_type
    df["model"] = (
        model.replace("_", " ")
        .replace("-", " ")
        .rstrip("-GGUF")
        .lstrip("NousResearch Hermes 3 Llama")
        .lstrip(".2")
        .lstrip(".3")
    )

df = pd.concat(dfs)
df

/tmp/ipykernel_1469200/2264217373.py:15: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df = pd.concat(dfs)


,model,f1_score,precision,recall,ged,normed_ged,response,cfg_name,zeroshot,stage,erl,generator,n_nodes,seed,erl_loaded
0,3B,0.000000,0.0,0.000000,6.0,-0.200000,The protein is a part of a larger protein doma...,UniProt,False,raw,"{""relations"":[{""entity"":""Reviewed protein (rec...",llama,3.0,0.0,relations=[EnrichedRelation(entity='Reviewed p...
1,3B,0.000000,0.0,0.000000,6.0,-0.200000,The protein is a part of a larger protein doma...,UniProt,False,constrained,"{""relations"":[{""entity"":""Reviewed protein (rec...",llama,3.0,0.0,relations=[EnrichedRelation(entity='Reviewed p...
2,3B,0.800000,1.0,0.666667,3.0,0.400000,The protein is a part of a larger protein doma...,UniProt,False,aligned,"{""relations"":[{""entity"":""Reviewed protein (rec...",llama,3.0,0.0,relations=[EnrichedRelation(entity='Reviewed p...
3,3B,0.800000,1.0,0.666667,3.0,0.400000,a Reviewed protein (records) has a domain with...,UniProt,False,raw,"{""relations"":[{""entity"":""Reviewed protein (rec...",templated,3.0,0.0,relations=[EnrichedRelation(entity='Reviewed p...
4,3B,0.000000,0.0,0.000000,45.0,-0.022727,a Reviewed protein (records) has a domain with...,UniProt,False,constrained,"{""relations"":[{""entity"":""Reviewed protein (rec...",templated,3.0,0.0,relations=[EnrichedRelation(entity='Reviewed p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4711,3B,0.000000,0.0,0.000000,209.0,-0.024510,"A winter sport player was born in a place, whi...",DBpedia,False,constrained,"{""relations"":[{""entity"":""winter sport Player"",...",llama,7.0,212.0,relations=[EnrichedRelation(entity='winter spo...
4712,3B,0.500000,0.6,0.428571,204.0,0.004878,"A winter sport player was born in a place, whi...",DBpedia,False,aligned,"{""relations"":[{""entity"":""winter sport Player"",...",llama,7.0,212.0,relations=[EnrichedRelation(entity='winter spo...
4713,3B,0.444444,1.0,0.285714,12.0,0.076923,"A figure skater was born in a village, had a f...",DBpedia,False,raw,"{""relations"":[{""entity"":""figure skater"",""relat...",llama,7.0,213.0,relations=[EnrichedRelation(entity='figure ska...
4714,3B,0.000000,0.0,0.000000,201.0,-0.025510,"A figure skater was born in a village, had a f...",DBpedia,False,constrained,"{""relations"":[{""entity"":""figure skater"",""relat...",llama,7.0,213.0,relations=[EnrichedRelation(entity='figure ska...


In [6]:
df

,model,f1_score,precision,recall,ged,normed_ged,response,cfg_name,zeroshot,stage,erl,generator,n_nodes,seed,erl_loaded
0,3B,0.000000,0.0,0.000000,6.0,-0.200000,The protein is a part of a larger protein doma...,UniProt,False,raw,"{""relations"":[{""entity"":""Reviewed protein (rec...",llama,3.0,0.0,relations=[EnrichedRelation(entity='Reviewed p...
1,3B,0.000000,0.0,0.000000,6.0,-0.200000,The protein is a part of a larger protein doma...,UniProt,False,constrained,"{""relations"":[{""entity"":""Reviewed protein (rec...",llama,3.0,0.0,relations=[EnrichedRelation(entity='Reviewed p...
2,3B,0.800000,1.0,0.666667,3.0,0.400000,The protein is a part of a larger protein doma...,UniProt,False,aligned,"{""relations"":[{""entity"":""Reviewed protein (rec...",llama,3.0,0.0,relations=[EnrichedRelation(entity='Reviewed p...
3,3B,0.800000,1.0,0.666667,3.0,0.400000,a Reviewed protein (records) has a domain with...,UniProt,False,raw,"{""relations"":[{""entity"":""Reviewed protein (rec...",templated,3.0,0.0,relations=[EnrichedRelation(entity='Reviewed p...
4,3B,0.000000,0.0,0.000000,45.0,-0.022727,a Reviewed protein (records) has a domain with...,UniProt,False,constrained,"{""relations"":[{""entity"":""Reviewed protein (rec...",templated,3.0,0.0,relations=[EnrichedRelation(entity='Reviewed p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4711,3B,0.000000,0.0,0.000000,209.0,-0.024510,"A winter sport player was born in a place, whi...",DBpedia,False,constrained,"{""relations"":[{""entity"":""winter sport Player"",...",llama,7.0,212.0,relations=[EnrichedRelation(entity='winter spo...
4712,3B,0.500000,0.6,0.428571,204.0,0.004878,"A winter sport player was born in a place, whi...",DBpedia,False,aligned,"{""relations"":[{""entity"":""winter sport Player"",...",llama,7.0,212.0,relations=[EnrichedRelation(entity='winter spo...
4713,3B,0.444444,1.0,0.285714,12.0,0.076923,"A figure skater was born in a village, had a f...",DBpedia,False,raw,"{""relations"":[{""entity"":""figure skater"",""relat...",llama,7.0,213.0,relations=[EnrichedRelation(entity='figure ska...
4714,3B,0.000000,0.0,0.000000,201.0,-0.025510,"A figure skater was born in a village, had a f...",DBpedia,False,constrained,"{""relations"":[{""entity"":""figure skater"",""relat...",llama,7.0,213.0,relations=[EnrichedRelation(entity='figure ska...


In [7]:
df["prompt_type"] = df["zeroshot"].apply(lambda x: "zero shot" if x else "one shot")
df.drop(columns=["zeroshot"], inplace=True)
df

,model,f1_score,precision,recall,ged,normed_ged,response,cfg_name,stage,erl,generator,n_nodes,seed,erl_loaded,prompt_type
0,3B,0.000000,0.0,0.000000,6.0,-0.200000,The protein is a part of a larger protein doma...,UniProt,raw,"{""relations"":[{""entity"":""Reviewed protein (rec...",llama,3.0,0.0,relations=[EnrichedRelation(entity='Reviewed p...,one shot
1,3B,0.000000,0.0,0.000000,6.0,-0.200000,The protein is a part of a larger protein doma...,UniProt,constrained,"{""relations"":[{""entity"":""Reviewed protein (rec...",llama,3.0,0.0,relations=[EnrichedRelation(entity='Reviewed p...,one shot
2,3B,0.800000,1.0,0.666667,3.0,0.400000,The protein is a part of a larger protein doma...,UniProt,aligned,"{""relations"":[{""entity"":""Reviewed protein (rec...",llama,3.0,0.0,relations=[EnrichedRelation(entity='Reviewed p...,one shot
3,3B,0.800000,1.0,0.666667,3.0,0.400000,a Reviewed protein (records) has a domain with...,UniProt,raw,"{""relations"":[{""entity"":""Reviewed protein (rec...",templated,3.0,0.0,relations=[EnrichedRelation(entity='Reviewed p...,one shot
4,3B,0.000000,0.0,0.000000,45.0,-0.022727,a Reviewed protein (records) has a domain with...,UniProt,constrained,"{""relations"":[{""entity"":""Reviewed protein (rec...",templated,3.0,0.0,relations=[EnrichedRelation(entity='Reviewed p...,one shot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4711,3B,0.000000,0.0,0.000000,209.0,-0.024510,"A winter sport player was born in a place, whi...",DBpedia,constrained,"{""relations"":[{""entity"":""winter sport Player"",...",llama,7.0,212.0,relations=[EnrichedRelation(entity='winter spo...,one shot
4712,3B,0.500000,0.6,0.428571,204.0,0.004878,"A winter sport player was born in a place, whi...",DBpedia,aligned,"{""relations"":[{""entity"":""winter sport Player"",...",llama,7.0,212.0,relations=[EnrichedRelation(entity='winter spo...,one shot
4713,3B,0.444444,1.0,0.285714,12.0,0.076923,"A figure skater was born in a village, had a f...",DBpedia,raw,"{""relations"":[{""entity"":""figure skater"",""relat...",llama,7.0,213.0,relations=[EnrichedRelation(entity='figure ska...,one shot
4714,3B,0.000000,0.0,0.000000,201.0,-0.025510,"A figure skater was born in a village, had a f...",DBpedia,constrained,"{""relations"":[{""entity"":""figure skater"",""relat...",llama,7.0,213.0,relations=[EnrichedRelation(entity='figure ska...,one shot


In [8]:
df_clean = df.dropna()
df_clean.describe()

,f1_score,precision,recall,ged,normed_ged,n_nodes,seed
count,20187.000000,20187.000000,20187.000000,20187.000000,20187.000000,20187.000000,20187.000000
mean,0.398872,0.413139,0.398947,10.837123,0.263886,4.921236,145.256948
std,0.418448,0.432493,0.424545,33.089750,0.356930,1.625566,85.969673
min,0.000000,0.000000,0.000000,0.000000,-0.384615,3.000000,0.000000
25%,0.000000,0.000000,0.000000,3.000000,0.000000,3.000000,71.000000
50%,0.333333,0.333333,0.285714,5.000000,0.133333,5.000000,144.000000
75%,0.800000,1.000000,0.857143,8.000000,0.500000,7.000000,218.000000
max,1.000000,1.000000,1.000000,406.000000,1.000000,10.000000,299.000000


In [9]:
df["erl_loaded"] = df["erl"].apply(qg.EnrichedEntitiesRelations.model_validate_json)
# df["response_loaded"] = df["response"].apply(lambda x: qg.EnrichedEntitiesRelations.model_validate_json(x) if isinstance(x, str) else None)
df["erl_graph"] = df["erl_loaded"].apply(graph_from_erl)
# df["response_graph"] = df["response"].apply(lambda x: graph_from_erl if x is not None else None)

In [10]:
# df["normed_ged"] = 1 - df["ged"] / df["erl_graph"].apply(
#     lambda x: (len(x.nodes) + len(x.nodes))
# )

In [28]:
df_clean["generator"].unique(), df_clean["model"].unique()

(array(['llama', 'templated', 'human'], dtype=object),
 array([' 3B ', '1 8B '], dtype=object))

In [11]:
means = df_clean.groupby(
    ["n_nodes", "prompt_type", "stage", "model", "generator", "cfg_name"]
)[["f1_score", "normed_ged"]].mean()
counts = df_clean.groupby(
    ["n_nodes", "stage", "prompt_type", "model", "generator", "cfg_name"]
)[["f1_score", "normed_ged"]].count()
means

f1_score  normed_ged
n_nodes prompt_type stage       model generator cfg_name                      
3.0     one shot    aligned      3B   human     DBpedia   0.633333    0.400000
                                      llama     BTO       0.595617    0.445091
                                                DBpedia   0.698514    0.458105
                                                DNB       0.174510    0.300700
                                                UniProt   0.562777    0.456491
...                                                            ...         ...
10.0    one shot    aligned     1 8B  human     DBpedia   0.734259    0.456558
                    constrained  3B   human     DBpedia   0.000000   -0.210623
                                1 8B  human     DBpedia   0.000000   -0.165163
                    raw          3B   human     DBpedia   0.652219    0.170426
                                1 8B  human     DBpedia   0.576190    0.177945

[96 rows x 2 columns]

In [12]:
counts

f1_score  normed_ged
n_nodes stage       prompt_type model generator cfg_name                      
3.0     aligned     one shot     3B   human     DBpedia          3           3
                                      llama     BTO            295         295
                                                DBpedia        290         290
                                                DNB             17          17
                                                UniProt        273         273
...                                                            ...         ...
10.0    aligned     one shot    1 8B  human     DBpedia          3           3
        constrained one shot     3B   human     DBpedia          3           3
                                1 8B  human     DBpedia          3           3
        raw         one shot     3B   human     DBpedia          3           3
                                1 8B  human     DBpedia          3           3

[96 rows x 2 columns]

In [13]:
column_map = {
    "f1_score": "$F_1$",
    "normed_ged": "$GED_{s}$",
    "stage": "Stage",
    "prompt_type": "Prompt Type",
    "model": "Version \\& Size",
    "n_nodes": "$k$",
    "generator": "Query Origin",
}

## Compare Query Origins on DBpedia with 8B model

In [14]:
query_gen_compare = df_clean[
    df_clean["model"].str.contains("8B")
    & df_clean["cfg_name"].str.contains("DBpedia")
    & df_clean["prompt_type"].str.contains("one shot")
]
grouped_query_gen_compare = query_gen_compare.groupby(
    ["n_nodes", "generator", "stage"]
)[["f1_score", "normed_ged"]].mean()
grouped_query_gen_compare = grouped_query_gen_compare.rename(
    columns=column_map
).rename_axis([column_map["n_nodes"], column_map["generator"], column_map["stage"]], axis=0)
with open("tables/compare_gen_query.tex", "w") as f:
    f.write(
        grouped_query_gen_compare.to_latex(
            float_format="%.2f",
            caption="Comparison of query generation methods for LLama 3.1 8B on DBpedia with one shot prompts.",
            label="tab:compare_gen_query",
        )
    )
grouped_query_gen_compare

$F_1$  $GED_{s}$
$k$  Query Origin Stage                           
3.0  human        aligned      0.266667   0.266667
                  constrained  0.000000   0.000000
                  raw          0.300000   0.200000
     llama        aligned      0.560965   0.440116
                  constrained  0.000000  -0.066244
                  raw          0.508317   0.249957
     templated    aligned      0.679895   0.568723
                  constrained  0.000000  -0.027883
                  raw          0.670223   0.442946
5.0  human        aligned      0.638889   0.431666
                  constrained  0.000000  -0.108393
                  raw          0.600000   0.107407
     llama        aligned      0.566096   0.334042
                  constrained  0.000000  -0.097322
                  raw          0.501306   0.174274
     templated    aligned      0.674380   0.538032
                  constrained  0.000000  -0.040298
                  raw          0.643058   0.394807
7.0  llama        aligned      0.557388   0.307650
                  constrained  0.000000  -0.097860
                  raw          0.485913   0.128287
     templated    aligned      0.677043   0.523685
                  constrained  0.000000  -0.040418
                  raw          0.633160   0.380653
10.0 human        aligned      0.734259   0.456558
                  constrained  0.000000  -0.165163
                  raw          0.576190   0.177945

## Compare models across Datasets

In [36]:
datasets = [
    "DBpedia",
    # "UniProt",
    "BTO",
    # "DNB",
]
grouped_dfs = {}
for dataset in datasets:
    model_compare = df_clean[df_clean["cfg_name"].str.strip() == dataset]
    grouped_model_compare = model_compare.groupby(
        [
            "n_nodes",
            "stage",
            "prompt_type",
            "model",
            # "generator",
        ]
    )[["f1_score", "normed_ged"]].mean()
    grouped_model_compare = grouped_model_compare.rename(
        columns=column_map
    ).rename_axis(
        [
            column_map["n_nodes"],
            column_map["stage"],
            column_map["prompt_type"],
            column_map["model"],
            # column_map["generator"],
        ],
        axis=0,
    )
    grouped_dfs[dataset] = grouped_model_compare
    with open(f"tables/compare_model_{dataset}.tex", "w") as f:
        f.write(
            grouped_model_compare.to_latex(
                float_format="%.2f",
                caption=f"Comparison of models for {dataset}",
                label=f"tab:compare_model_{dataset}",
                # sparsify=False
            )
        )
dataset_keys = list(grouped_dfs.keys())
merged_grouped: pd.DataFrame = grouped_dfs[dataset_keys[0]].copy()
merged_grouped.rename(
    columns={
        "$F_1$": f"$F_1$_{dataset_keys[0]}",
        "$GED_{s}$": f"$GED_{{s}}$_{dataset_keys[0]}",
    },
    inplace=True,
)
for dataset, grouped_df in list(grouped_dfs.items())[1:]:
    print(f"Merging {dataset} into the grouped DataFrame")
    merged_grouped = pd.merge(
        merged_grouped,
        grouped_dfs[dataset].rename(
            columns={
                "$F_1$": f"$F_1$_{dataset}",
                "$GED_{s}$": f"$GED_{{s}}$_{dataset}",
            }
        ),
        on=[
            "$k$",
            "Stage",
            "Prompt Type",
            "Version \\& Size",
            # "Query Origin",
        ],
        # suffixes=(None, f"_{dataset}"),
    )
rename_map = {f"$F_1$_{dataset}": f"{dataset} $F_1$" for dataset in datasets} | {
    f"$GED_{{s}}$_{dataset}": f"{dataset} $GED_{{s}}$" for dataset in datasets
}
merged_grouped.rename(
    columns=rename_map,
    inplace=True,
)
with open("tables/compare_model_merged.tex", "w") as f:
    f.write(
        merged_grouped.to_latex(
            float_format="%.2f",
            caption="Comparison of models for DBpedia and UniProt",
            label="tab:compare_model_merged",
        )
    )

Merging BTO into the grouped DataFrame


In [37]:
grouped_dfs

{'DBpedia':                                                  $F_1$  $GED_{s}$
 $k$  Stage       Prompt Type Version \& Size                     
 3.0  aligned     one shot     3B              0.747539   0.506308
                              1 8B             0.618738   0.503323
      constrained one shot     3B              0.000000  -0.045725
                              1 8B             0.000000  -0.046793
      raw         one shot     3B              0.708324   0.442749
                              1 8B             0.587941   0.345873
 5.0  aligned     one shot     3B              0.705811   0.404506
                              1 8B             0.620985   0.437241
      constrained one shot     3B              0.000000  -0.066838
                              1 8B             0.000000  -0.068671
      raw         one shot     3B              0.668095   0.351033
                              1 8B             0.573178   0.284954
 7.0  aligned     one shot     3B              0.69

In [38]:
merged_grouped

DBpedia $F_1$  DBpedia $GED_{s}$  \
$k$ Stage       Prompt Type Version \& Size                                     
3.0 aligned     one shot    3B                    0.747539           0.506308   
    constrained one shot    3B                    0.000000          -0.045725   
    raw         one shot    3B                    0.708324           0.442749   
5.0 aligned     one shot    3B                    0.705811           0.404506   
    constrained one shot    3B                    0.000000          -0.066838   
    raw         one shot    3B                    0.668095           0.351033   
7.0 aligned     one shot    3B                    0.697654           0.382295   
    constrained one shot    3B                    0.000000          -0.079606   
    raw         one shot    3B                    0.646449           0.298021   

                                             BTO $F_1$  BTO $GED_{s}$  
$k$ Stage       Prompt Type Version \& Size                            
3.0 aligned     one shot    3B                0.694183       0.514507  
    constrained one shot    3B                0.000000      -0.001362  
    raw         one shot    3B                0.511569       0.461125  
5.0 aligned     one shot    3B                0.692153       0.518314  
    constrained one shot    3B                0.000000      -0.005766  
    raw         one shot    3B                0.502726       0.454024  
7.0 aligned     one shot    3B                0.697552       0.514315  
    constrained one shot    3B                0.000000      -0.004838  
    raw         one shot    3B                0.505157       0.453439

In [39]:
df_clean["cfg_name"].str.strip() == "DBpedia"

0       False
1       False
2       False
3       False
4       False
        ...  
4711     True
4712     True
4713     True
4714     True
4715     True
Name: cfg_name, Length: 20187, dtype: bool